<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорты

In [1]:
# !pip install pymystem3 -q
# !pip install transformers -q

In [2]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertConfig, BertModel
import transformers
import nltk
import logging
import warnings
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.utils import resample
from pymystem3 import Mystem
import re

# Оптимизация гиперпараметров
import optuna
# from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Прочее
from tqdm import tqdm, notebook
tqdm.pandas()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# Отключение предупреждений
warnings.filterwarnings("ignore")

e:\my_github\yandex_practicum_ds\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# подгрузки
# bert
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
config = BertConfig.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", config=config)

# stopwords
nltk.download('stopwords')
english_stopwords = nltk_stopwords.words('english')

# cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # обязательно для инференса
logger.info(device)



### Константы

In [ ]:
MAX_LENGTH = 512 # макс длина для bert
BATCH_SIZE = 100
CV = 5
N_OPTUNA = 5
TEST_SIZE = 0.2
RANDOM_STATE = 20

In [ ]:
# data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])
df = pd.read_csv('./data/toxic_comments.csv', index_col=[0])
display(df.head())

### Подготовка текста

In [ ]:
m = Mystem()
corpus = df['text'].values.astype('U')

In [ ]:
# почистим текст
def fast_clean(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|@\w+|#\w+", "", text)
    text = re.sub(r"[^a-z0-9 ]", " ", text)

    tokens = text.split()
    tokens = [w for w in tokens if w not in english_stopwords]

    return " ".join(tokens)

df['text_clean'] = df['text'].progress_apply(fast_clean)

In [ ]:
display(df.sample(5))

In [ ]:
# приведем к лемам
# попробовал в рамках тест - более суток обработка, ну ее... пусть пока лежит закомменченное
# либо я что-то не так делаю...
# def lemmatize(text):
#     if not isinstance(text, str):
#         return ""
#     return " ".join(m.lemmatize(text))

# df['lema_text'] = df['text_clean'].progress_apply(lemmatize)

In [ ]:
# N = 0 -> берет все данные
N = 0

# разбиваем по классам
df_0 = df[df['toxic'] == 0]
df_1 = df[df['toxic'] == 1]

if N == 0:
    df_0_sample = df_0
    df_1_sample = df_1
else:
    n_samples = N // 2  # половина от N для каждого класса
    df_0_sample = resample(df_0, n_samples=n_samples, replace=False, random_state=RANDOM_STATE)
    df_1_sample = resample(df_1, n_samples=n_samples, replace=False, random_state=RANDOM_STATE)

# объединяем и перемешиваем
df_sample = pd.concat([df_0_sample, df_1_sample]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

In [ ]:
all_embeddings = []

for i in tqdm(range(0, len(df_sample), BATCH_SIZE), desc="Tokenizing & Embedding"):
    batch_texts = df_sample['text_clean'].iloc[i:i+BATCH_SIZE].tolist()
    
    # батчевая токенизация
    encoded = tokenizer(
        batch_texts,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    
    # переносим на GPU
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    # инференс через BERT
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # CLS-токен, переводим в numpy (CPU)
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    all_embeddings.append(cls_embeddings)

# объединяем все батчи в один массив
features = np.concatenate(all_embeddings, axis=0)

## Обучение

In [ ]:
X = features
y = df_sample['toxic']
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

In [ ]:
def objective(trial):
    model_name = trial.suggest_categorical("model", ["LogisticRegression", "DecisionTree", "RandomForest", "LGBM"])

    if model_name == "LogisticRegression":
        C = trial.suggest_float("C", 1e-3, 10.0, log=True)
        clf = LogisticRegression(C=C, max_iter=1000, class_weight="balanced", n_jobs=-1)
    
    elif model_name == "DecisionTree":
        max_depth = trial.suggest_int("max_depth", 2, 10)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        clf = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, class_weight="balanced")
    
    elif model_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        max_depth = trial.suggest_int("max_depth", 2, 10)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        clf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            class_weight="balanced",
            n_jobs=-1
        )
    
    else:  # LGBM
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        max_depth = trial.suggest_int("max_depth", 2, 10)
        learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.3, log=True)
        clf = LGBMClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            class_weight="balanced",
            n_jobs=-1,
            verbose=-1
        )

    # Кросс-валидация на трейне
    cv = StratifiedKFold(n_splits=CV, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring="f1")
    
    # компактный вывод F1 для этой модели
    logger.info(f"{model_name}: средний F1 на трейне CV = {scores.mean():.4f}")
    
    return scores.mean()

# Запуск Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_OPTUNA)  # можно увеличить

# финальная лучшая модель
best_trial = study.best_trial
best_model_name = best_trial.params.pop("model")

if best_model_name == "LogisticRegression":
    final_model = LogisticRegression(**best_trial.params, max_iter=1000, class_weight="balanced", n_jobs=-1)
elif best_model_name == "DecisionTree":
    final_model = DecisionTreeClassifier(**best_trial.params, class_weight="balanced")
elif best_model_name == "RandomForest":
    final_model = RandomForestClassifier(**best_trial.params, class_weight="balanced", n_jobs=-1)
else:
    final_model = LGBMClassifier(**best_trial.params, class_weight="balanced", n_jobs=-1, verbose=-1)

# обучение на всём трейне
final_model.fit(X_train, y_train)

# оценка на тесте
y_pred = final_model.predict(X_val)
test_f1 = f1_score(y_val, y_pred)

logger.info(f"Лучшая модель: {best_model_name}")
logger.info(f"Параметры: {best_trial.params}")
logger.info(f"F1 на тесте: {test_f1:.4f}")

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны